In [2]:
import os

In [6]:
%pwd

'c:\\Users\\91812\\Desktop\\DATASCIENCE\\End_to_End_Project\\Chiken_disease_End_to_End'

In [5]:
os.chdir('../')

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_dir: Path
    checkpoint_model_filepath: Path

In [10]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml , create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self ,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:

        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir) , Path(config.tensorboard_root_log_dir)])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config


In [19]:
import os
import tensorflow as tf
import time
from pathlib import Path

In [22]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    @property
    def create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_dir , 
            f"tb_logs_at_{timestamp}"
        )

        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)

    @property
    def create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath= str(self.config.checkpoint_model_filepath), # in this path is not allowed so for this we have converted windowPath to strinf
            save_best_only = True
        )


    def get_tb_ckpt_callbacks(self):

        return [
            self.create_tb_callbacks,
            self.create_ckpt_callbacks
        ]




In [23]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

[2023-11-01 16:23:47,567: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-01 16:23:47,572: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-01 16:23:47,574: INFO: common: created directory at : artifacts]
[2023-11-01 16:23:47,575: INFO: common: created directory at : artifacts\prepare_callbacks\checkpoint_dir]
[2023-11-01 16:23:47,578: INFO: common: created directory at : artifacts\prepare_callbacks\tensorboard_log_dir]
